In [1]:
# Install necessary packages
!pip install ultralytics roboflow opencv-python torch torchvision numpy matplotlib

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     -------------------------- ----------- 41.0/57.7 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 57.7/57.7 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   - -------------------------------------- 30.7/914.9 kB 1.4 MB/s eta 0:00:01
   -- ------------------------------------ 61.4/914.9 kB 656.4 kB/s eta 0:00:02
   ---- --------------------------------- 112.6/914.9 kB 819.2 kB/s eta 0:00:01
   ------ ------------------------------- 153.6/914.9 kB 833.5 kB/s eta 0:00:01
   -------- ----------------------------- 204.8/914.9 kB 888.4 kB/s eta 0:00:01
   --------- ---------------------------- 235.5/914.9 kB 901.1 kB/s eta 0:00:01
   ----------- -------------------------- 286.7/914.9 kB 930.9 kB/s eta 0:00:01
   --------------- ------------------------ 358.4/914.9 kB 1

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\kiran\AppData\Roaming\Python\Python312\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\kiran\AppData\Roaming\Python\Python312\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\kiran\AppData\Roaming\Python\Python312\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\kiran\AppData\Roaming\Python\Python312\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python312\Lib\http\client.py", line 472, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python312\Lib\s

   ------------------------- ------------ 139.0/204.1 MB 270.9 kB/s eta 0:04:01
   ------------------------- ------------ 139.0/204.1 MB 270.9 kB/s eta 0:04:01
   ------------------------- ------------ 139.0/204.1 MB 270.9 kB/s eta 0:04:01
   ------------------------- ------------ 139.0/204.1 MB 270.9 kB/s eta 0:04:01
   ------------------------- ------------ 139.0/204.1 MB 270.9 kB/s eta 0:04:01
   ------------------------- ------------ 139.0/204.1 MB 270.9 kB/s eta 0:04:01
   ------------------------- ------------ 139.0/204.1 MB 270.9 kB/s eta 0:04:01
   ------------------------- ------------ 139.0/204.1 MB 270.9 kB/s eta 0:04:01
   ------------------------- ------------ 139.0/204.1 MB 270.9 kB/s eta 0:04:01
   ------------------------- ------------ 139.0/204.1 MB 270.9 kB/s eta 0:04:01
   ------------------------- ------------ 139.0/204.1 MB 270.9 kB/s eta 0:04:01
   ------------------------- ------------ 139.0/204.1 MB 270.9 kB/s eta 0:04:01
   ------------------------- -----------

In [1]:

# Import required libraries
from roboflow import Roboflow
from ultralytics import YOLO
import torch
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt


optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\kiran\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Check if CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
# Initialize Roboflow and download dataset
rf = Roboflow(api_key="V6l7eBqVkekBX5xgVvna")
project = rf.workspace("college-if44x").project("on-tree-mature-coconut-fruit-detection")
version = project.version(1)
dataset = version.download("yolov8")
dataset_path = dataset.location

In [ ]:
# Data augmentation
augmentation = {
    "flipud": 0.5,  # Vertical flip probability
    "fliplr": 0.5,  # Horizontal flip probability
    "mosaic": 1.0,  # Mosaic augmentation
    "mixup": 0.2,   # Mixup augmentation
    "hsv_h": 0.015,  # Hue shift
    "hsv_s": 0.7,    # Saturation shift
    "hsv_v": 0.4     # Value (brightness) shift
}


In [ ]:
# Train YOLOv8 model
model = YOLO("yolov8n.yaml")  # Choose 'n', 's', 'm', or 'l' based on your needs

model.train(
    data=os.path.join(dataset_path, "data.yaml"),
    epochs=100,       # Increased for better training
    batch=16,         # Keep batch size
    imgsz=640,        # Image size
    device=device,
    name="coconut_maturity_model",
    lr0=0.01,         # Initial learning rate
    lrf=0.0001,       # Final learning rate
    weight_decay=0.0005,  # Regularization
    patience=10,      # Early stopping patience
    dropout=0.2,      # Prevent overfitting
    augment=True      # Enable augmentations
)


In [ ]:
# Evaluate the model
metrics = model.val()
print("Evaluation Metrics:", metrics)

In [ ]:
# Extract evaluation metrics correctly
precision = metrics.box.map50    # mAP@50
recall = metrics.box.map         # mAP@50-95 (average precision)
map50 = metrics.box.mp           # Mean precision
map50_95 = metrics.box.mr        # Mean recall

# Display evaluation results
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}")
print(f"mAP@50: {map50:.4f}, mAP@50-95: {map50_95:.4f}")


In [ ]:
# Run inference on a test image
test_image_path = "/content/On-tree-mature-coconut-fruit-detection-1/test/images/00003_jpg.rf.685ee7b127bd0896095d95bc5b656151.jpg"
results = model(test_image_path)  # This returns a list of results

# Access the first result
result = results[0]

# Display the image with predictions
result.show()

# Save the output image with predictions
result.save("output2.jpg")


In [ ]:
# Save the trained model
model_path = "coconut_maturity_model_new.pt"
model.export(format="torchscript")  # You can also use 'onnx', 'coreml', or 'tflite' if needed
model.save(model_path)

print(f"Model saved as {model_path}")

In [ ]:
# Real-time Coconut Detection Using Webcam
def detect_coconut_webcam(model):
    cap = cv2.VideoCapture(0)  # Open webcam
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        results = model(frame)
        result = results[0]
        annotated_frame = result.plot()

        cv2.imshow("Coconut Detection", annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
            break

    cap.release()
    cv2.destroyAllWindows()

# Run webcam detection
detect_coconut_webcam(model)

In [ ]:
import sys
import subprocess

# Upgrade pip
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])

# Install Hugging Face Hub
subprocess.run([sys.executable, "-m", "pip", "install", "huggingface_hub"])


In [ ]:
from huggingface_hub import notebook_login
notebook_login()  # This will prompt you to enter your Hugging Face token

In [ ]:
from huggingface_hub import create_repo

repo_id = "Siva-67/coconut_maturity_model_new"  # Replace with your Hugging Face username/repo name
create_repo(repo_id, exist_ok=True)


In [ ]:
from huggingface_hub import HfApi

api = HfApi()
model_path = "coconut_maturity_model_new.pt"  # Replace with your actual model file

api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="coconut_maturity_model_new.pt",
    repo_id=repo_id
)

print("Model uploaded successfully!")


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [ ]:
!git clone https://github.com/siva-netizen/yolov8--coconut_maturity_lvl.git
%cd yolov8--coconut_maturity_lvl

In [ ]:
!ls

In [ ]:
%cd /content/yolov8--coconut_maturity_lvl



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r /content/yolov8--coconut_maturity_lvl /content/drive/MyDrive/

In [ ]:
!ls /content/drive/MyDrive/yolov8--coconut_maturity_lvl

In [ ]:
!ls /content/yolov8--coconut_maturity_lvl